# REDES NEURAIS ARTIFICIAIS
## Evaluate Improve e Tunning

# Parte 1 - Pré-processamento dos Dados

### Importar as libs

In [3]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [4]:
tf.__version__

'2.8.0'

### Importar o dataset

In [5]:
dataset = pd.read_csv('/content/Churn_Modelling.csv')
dataset.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [6]:
X = dataset.iloc[:, 3:13].values
y = dataset.iloc[:, 13].values
print("X >>\n",X)
print("y >>\n",y)

X >>
 [[619 'France' 'Female' ... 1 1 101348.88]
 [608 'Spain' 'Female' ... 0 1 112542.58]
 [502 'France' 'Female' ... 1 0 113931.57]
 ...
 [709 'France' 'Female' ... 0 1 42085.58]
 [772 'Germany' 'Male' ... 1 0 92888.52]
 [792 'France' 'Female' ... 1 0 38190.78]]
y >>
 [1 0 1 ... 1 1 0]


### Transformando os dados categóricos


In [7]:
# Label Encoding the "Gender" column
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
X[:, 2] = le.fit_transform(X[:, 2])
print(X)
# One Hot Encoding the "Geography" column
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [1])], remainder='passthrough')
X = np.array(ct.fit_transform(X))
print(X)

[[619 'France' 0 ... 1 1 101348.88]
 [608 'Spain' 0 ... 0 1 112542.58]
 [502 'France' 0 ... 1 0 113931.57]
 ...
 [709 'France' 0 ... 0 1 42085.58]
 [772 'Germany' 1 ... 1 0 92888.52]
 [792 'France' 0 ... 1 0 38190.78]]
[[1.0 0.0 0.0 ... 1 1 101348.88]
 [0.0 0.0 1.0 ... 0 1 112542.58]
 [1.0 0.0 0.0 ... 1 0 113931.57]
 ...
 [1.0 0.0 0.0 ... 0 1 42085.58]
 [0.0 1.0 0.0 ... 1 0 92888.52]
 [1.0 0.0 0.0 ... 1 0 38190.78]]


# Dividindo o dataset em conjunto de treinamento e conjunto de teste

In [8]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

# Feature Scaling

In [9]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [10]:
print("X_train shape: ",np.shape(X_train))
print("X_test shape: ",X_test.shape)

X_train shape:  (8000, 12)
X_test shape:  (2000, 12)


---
# Parte 2 -Vamos construir uma ANN!


### Initializing the ANN

In [11]:
ann = tf.keras.models.Sequential()

### Adding the input layer and the first hidden layer

In [12]:
ann.add(tf.keras.layers.Dense(units=12, activation='gelu'))

### Adding the second hidden layer

In [13]:
ann.add(tf.keras.layers.Dense(units=6, activation='swish'))

### Adding the output layer

In [14]:
ann.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

## Part 3 - Training the ANN

### Compiling the ANN

In [15]:
ann.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

### Training the ANN on the Training set

In [16]:
ann.fit(X_train, y_train, batch_size = 32, epochs = 100)

Epoch 1/100
250/250 [==============================] - 1s 2ms/step - loss: 0.5187 - accuracy: 0.7700
Epoch 2/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4240 - accuracy: 0.8195
Epoch 3/100
250/250 [==============================] - 0s 2ms/step - loss: 0.3989 - accuracy: 0.8342
Epoch 4/100
250/250 [==============================] - 0s 2ms/step - loss: 0.3821 - accuracy: 0.8429
Epoch 5/100
250/250 [==============================] - 0s 2ms/step - loss: 0.3712 - accuracy: 0.8482
Epoch 6/100
250/250 [==============================] - 0s 2ms/step - loss: 0.3648 - accuracy: 0.8503
Epoch 7/100
250/250 [==============================] - 0s 2ms/step - loss: 0.3597 - accuracy: 0.8525
Epoch 8/100
250/250 [==============================] - 0s 2ms/step - loss: 0.3572 - accuracy: 0.8529
Epoch 9/100
250/250 [==============================] - 1s 2ms/step - loss: 0.3549 - accuracy: 0.8539
Epoch 10/100
250/250 [==============================] - 0s 2ms/step - loss: 0.3532 - accura

# Parte 3 - Fazendo predições e avaliando o modelo

## Prevendo os resultados com o conjunto de testes

In [17]:
y_pred = ann.predict(X_test)
y_pred

array([[0.23674   ],
       [0.23775238],
       [0.05229819],
       ...,
       [0.11742491],
       [0.16930044],
       [0.1828239 ]], dtype=float32)

In [18]:
y_pred = (y_pred > 0.5)
y_pred

array([[False],
       [False],
       [False],
       ...,
       [False],
       [False],
       [False]])

# Criando uma Confusion Matrix

In [19]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
print(cm)

[[1517   78]
 [ 203  202]]


In [20]:
Tp, Fp, Fn, Tn = cm.reshape(1, -1, order='F').squeeze()
accuracy = (Tp+Tn) / (Tp+Fp+Tn+Fn)
precision = (Tp) / (Tp+Fp)
recall = (Tp) / (Tp+Fn)
f1_score = 2 * (precision*recall) / (precision+recall)
metrics_names = ["Accuracy", "Precision", "Recall", "F1 Score"]
metrics = [accuracy, precision, recall, f1_score]
session_results = pd.DataFrame(metrics, index=metrics_names)
print(session_results)

                  0
Accuracy   0.859500
Precision  0.881977
Recall     0.951097
F1 Score   0.915234


---
# Avaliando o Modelo

In [21]:
# Evaluating the ANN
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [22]:
def build_classifier():
    classifier = Sequential()
    classifier.add(Dense(units = 12, kernel_initializer = 'uniform', activation = 'gelu', input_dim = 12))
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'swish'))
    classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))
    classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
    return classifier

In [23]:
classifier = KerasClassifier(build_fn = build_classifier, batch_size = 10, epochs = 10)
accuracies = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10, n_jobs = -1)
mean = accuracies.mean()
variance = accuracies.std()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  """Entry point for launching an IPython kernel.


In [24]:
print(mean,variance)

0.8606249928474426 0.01097226742375363


# Melhorando o modelo

In [25]:
# Improving the ANN
# Dropout Regularization to reduce overfitting if needed
from tensorflow.keras.layers import Dropout

def build_classifier():
    classifier = Sequential()
    classifier.add(Dense(units = 12, kernel_initializer = 'uniform', activation = 'gelu', input_dim = 12))
    classifier.add(Dropout(rate = 0.1))
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'swish'))
    classifier.add(Dropout(rate = 0.1))
    classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))
    classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
    return classifier

In [26]:
classifier = KerasClassifier(build_fn = build_classifier, batch_size = 10, epochs = 10)
accuracies = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10, n_jobs = -1)
mean = accuracies.mean()
variance = accuracies.std()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  """Entry point for launching an IPython kernel.


In [27]:
print(mean,variance)

0.8601249933242798 0.01013117074719109


# Tuning the ANN

In [31]:
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

def build_classifier(optimizer):
    classifier = Sequential()
    classifier.add(Dense(units = 12, kernel_initializer = 'uniform', activation = 'gelu', input_dim = 12))
    classifier.add(Dropout(rate = 0.1))
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'swish'))
    classifier.add(Dropout(rate = 0.1))
    classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))
    classifier.compile(optimizer = optimizer, loss = 'binary_crossentropy', metrics = ['accuracy'])
    return classifier

classifier = KerasClassifier(build_fn = build_classifier)

parameters = {
    'batch_size': [10, 25, 32],
    'epochs': [50, 100],
    'optimizer': ['adam', 'rmsprop']
}

grid_search = GridSearchCV(estimator = classifier,
                           scoring = 'accuracy',
                           param_grid = parameters,
                           cv = 10)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  app.launch_new_instance()


In [32]:
%%time

grid_search = grid_search.fit(X_train, y_train)

A saída de streaming foi truncada nas últimas 5000 linhas.
Epoch 2/50
225/225 [==============================] - 1s 3ms/step - loss: 0.4517 - accuracy: 0.8082
Epoch 3/50
225/225 [==============================] - 1s 2ms/step - loss: 0.4285 - accuracy: 0.8167
Epoch 4/50
225/225 [==============================] - 1s 3ms/step - loss: 0.4235 - accuracy: 0.8215
Epoch 5/50
225/225 [==============================] - 1s 3ms/step - loss: 0.4152 - accuracy: 0.8272
Epoch 6/50
225/225 [==============================] - 1s 2ms/step - loss: 0.4074 - accuracy: 0.8300
Epoch 7/50
225/225 [==============================] - 1s 3ms/step - loss: 0.3980 - accuracy: 0.8360
Epoch 8/50
225/225 [==============================] - 1s 2ms/step - loss: 0.3882 - accuracy: 0.8397
Epoch 9/50
225/225 [==============================] - 1s 3ms/step - loss: 0.3769 - accuracy: 0.8432
Epoch 10/50
225/225 [==============================] - 1s 2ms/step - loss: 0.3689 - accuracy: 0.8482
Epoch 11/50
225/225 [===================

In [33]:
print('Best params:\n',grid_search.best_params_,'\nBest accuracy:\n',grid_search.best_score_)

Best params:
 {'batch_size': 10, 'epochs': 100, 'optimizer': 'adam'} 
Best accuracy:
 0.8633750000000001


# Halving Grid Search

In [40]:
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingGridSearchCV

def build_classifier(optimizer):
    classifier = Sequential()
    classifier.add(Dense(units = 12, kernel_initializer = 'uniform', activation = 'gelu', input_dim = 12))
    classifier.add(Dropout(rate = 0.1))
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'swish'))
    classifier.add(Dropout(rate = 0.1))
    classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))
    classifier.compile(optimizer = optimizer, loss = 'binary_crossentropy', metrics = ['accuracy'])
    return classifier

classifier = KerasClassifier(build_fn = build_classifier)

parameters = {
    'batch_size': [10, 25, 32],
    'epochs': [50, 100],
    'optimizer': ['adam', 'rmsprop']
}

halving_cv = HalvingGridSearchCV(estimator = classifier,
                                  param_grid = parameters,
                                  scoring="accuracy",  
                                  min_resources="exhaust", 
                                  factor=3,
                                  cv = 10, n_jobs = -2)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.


In [41]:
%%time
halving_cv = halving_cv.fit(X_train, y_train)

A saída de streaming foi truncada nas últimas 5000 linhas.
Epoch 2/100
96/96 [==============================] - 0s 3ms/step - loss: 0.5511 - accuracy: 0.8031
Epoch 3/100
96/96 [==============================] - 0s 3ms/step - loss: 0.4476 - accuracy: 0.8068
Epoch 4/100
96/96 [==============================] - 0s 4ms/step - loss: 0.4301 - accuracy: 0.8114
Epoch 5/100
96/96 [==============================] - 0s 3ms/step - loss: 0.4281 - accuracy: 0.8235
Epoch 6/100
96/96 [==============================] - 0s 3ms/step - loss: 0.4187 - accuracy: 0.8231
Epoch 7/100
96/96 [==============================] - 0s 3ms/step - loss: 0.4173 - accuracy: 0.8235
Epoch 8/100
96/96 [==============================] - 0s 3ms/step - loss: 0.4117 - accuracy: 0.8315
Epoch 9/100
96/96 [==============================] - 0s 3ms/step - loss: 0.4097 - accuracy: 0.8323
Epoch 10/100
96/96 [==============================] - 0s 3ms/step - loss: 0.4055 - accuracy: 0.8344
Epoch 11/100
96/96 [=============================

In [42]:
print('Best params:\n',halving_cv.best_params_,'\nBest accuracy:\n',halving_cv.best_score_)

Best params:
 {'batch_size': 25, 'epochs': 100, 'optimizer': 'adam'} 
Best accuracy:
 0.8624530663329161


# FIM

> Professor Diego Dorgam  
> [@diegodorgam](https://twitter.com/diegodorgam)